In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import logging

# Setup logging for better traceability
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# User-Agent rotation list
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    # Add more user agents as needed
]

# Proxy list (example, fill with real proxy data)
proxies = [
    {"http": "http://proxy1.com:8080", "https": "http://proxy1.com:8080"},
    {"http": "http://proxy2.com:8080", "https": "http://proxy2.com:8080"},
    # Add more proxies as needed
]

# Function to fetch a random user agent
def get_random_user_agent():
    return random.choice(user_agents)

# Function to fetch a random proxy
def get_random_proxy():
    return random.choice(proxies) if proxies else None

# Container to store the reviews
hotel_reviews_container = []

# Function to extract reviews from a single hotel page
def extract_hotel_reviews(hotel_url, num_reviews):
    reviews = []
    current_page = 0
    reviews_extracted = 0
    retries = 3  # Number of retries on failure

    while reviews_extracted < num_reviews:
        page_url = f"{hotel_url}-or{current_page*5}"
        headers = {"User-Agent": get_random_user_agent()}
        proxy = get_random_proxy()

        for attempt in range(retries):
            try:
                logging.info(f"Fetching page: {page_url} with headers: {headers} and proxy: {proxy}")
                response = requests.get(page_url, headers=headers, proxies=proxy, timeout=10)
                
                if response.status_code == 429:
                    logging.warning(f"Rate limit hit. Sleeping for 60 seconds...")
                    time.sleep(60)
                    continue

                if response.status_code != 200:
                    logging.error(f"Failed to retrieve data for page: {page_url} (status code: {response.status_code})")
                    break

                soup = BeautifulSoup(response.text, 'html.parser')

                # Update this selector according to TripAdvisor's current HTML structure
                review_blocks = soup.find_all('div', class_='review-container')

                if not review_blocks:
                    logging.warning(f"No review blocks found on page: {page_url}")
                    break

                for block in review_blocks:
                    review_text = block.find('q', class_='hotels-review-list-parts-ExpandableReview__reviewText--3oMkH')
                    if review_text:
                        reviews.append(review_text.text.strip())
                        reviews_extracted += 1
                        if reviews_extracted >= num_reviews:
                            return reviews

                time.sleep(random.uniform(1, 3))  # Random delay to avoid detection
                current_page += 1
                break  # Exit the retry loop if successful

            except requests.exceptions.RequestException as e:
                logging.error(f"Request failed for {page_url}, attempt {attempt + 1}/{retries}: {e}")
                time.sleep(5)  # Delay between retries
                if attempt == retries - 1:
                    logging.error(f"Failed to fetch {page_url} after {retries} attempts. Skipping...")
                    return reviews

    return reviews


# Function to fetch reviews for top N hotels
def extract_top_hotels_reviews(hotel_urls, reviews_per_hotel):
    global hotel_reviews_container
    
    for index, hotel_url in enumerate(hotel_urls):
        logging.info(f"Extracting reviews for hotel {index + 1}/{len(hotel_urls)}: {hotel_url}")
        reviews = extract_hotel_reviews(hotel_url, reviews_per_hotel)
        hotel_reviews_container.append({
            "hotel_url": hotel_url,
            "reviews": reviews
        })
        logging.info(f"Extracted {len(reviews)} reviews from {hotel_url}")

        # Simulate a delay between each hotel to avoid overloading the server
        time.sleep(random.uniform(2, 5))


# Example hotel URLs (In practice, these should be fetched dynamically based on top-rated hotels)
hotel_urls = [
    'https://www.tripadvisor.com/Hotel_Review-g60763-d123456-Reviews-The_Hotel_New_York_City_New_York.html',
    'https://www.tripadvisor.com/Hotel_Review-g60898-d123456-Reviews-The_Hotel_Atlanta_Georgia.html',
    # Add more hotel URLs...
]

# Define the number of hotels and reviews per hotel
num_hotels = 150
reviews_per_hotel = 150

# Fetch reviews for top hotels
extract_top_hotels_reviews(hotel_urls[:num_hotels], reviews_per_hotel)

# Save or process the data as needed
for hotel in hotel_reviews_container:
    logging.info(f"Hotel: {hotel['hotel_url']}")
    logging.info(f"First 5 reviews: {hotel['reviews'][:5]}")

2025-05-13 11:16:49,733 - INFO - Extracting reviews for hotel 1/2: https://www.tripadvisor.com/Hotel_Review-g60763-d123456-Reviews-The_Hotel_New_York_City_New_York.html
2025-05-13 11:16:49,733 - INFO - Fetching page: https://www.tripadvisor.com/Hotel_Review-g60763-d123456-Reviews-The_Hotel_New_York_City_New_York.html-or0 with headers: {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'} and proxy: {'http': 'http://proxy1.com:8080', 'https': 'http://proxy1.com:8080'}
2025-05-13 11:17:10,070 - ERROR - Request failed for https://www.tripadvisor.com/Hotel_Review-g60763-d123456-Reviews-The_Hotel_New_York_City_New_York.html-or0, attempt 1/3: HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Max retries exceeded with url: /Hotel_Review-g60763-d123456-Reviews-The_Hotel_New_York_City_New_York.html-or0 (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnectio

In [4]:
# new 


In [6]:
!pip install requests beautifulsoup4 pandas openpyxl


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# User-Agent rotation list
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
]

# Optional proxies (add real proxies if needed)
proxies = []

# Helper functions
def get_random_user_agent():
    return random.choice(user_agents)

def get_random_proxy():
    return random.choice(proxies) if proxies else None

# Core function to extract reviews
def extract_hotel_reviews(hotel_url, num_reviews):
    reviews = []
    current_page = 0
    reviews_extracted = 0
    retries = 3

    while reviews_extracted < num_reviews:
        page_url = f"{hotel_url}-or{current_page * 5}"
        headers = {"User-Agent": get_random_user_agent()}
        proxy = get_random_proxy()

        for attempt in range(retries):
            try:
                logging.info(f"Fetching: {page_url}")
                response = requests.get(page_url, headers=headers, proxies=proxy, timeout=10)

                if response.status_code == 429:
                    logging.warning("Rate limit hit. Sleeping for 60 seconds...")
                    time.sleep(60)
                    continue

                if response.status_code != 200:
                    logging.error(f"HTTP Error {response.status_code}")
                    break

                soup = BeautifulSoup(response.text, 'html.parser')
                review_blocks = soup.find_all('q', class_='QewHA H4 _a')  # Update this if it changes

                if not review_blocks:
                    logging.warning(f"No reviews found on page {page_url}")
                    break

                for block in review_blocks:
                    text = block.get_text(strip=True)
                    reviews.append(text)
                    reviews_extracted += 1
                    if reviews_extracted >= num_reviews:
                        return reviews

                current_page += 1
                time.sleep(random.uniform(1, 3))
                break

            except requests.exceptions.RequestException as e:
                logging.error(f"Attempt {attempt+1} failed for {page_url}: {e}")
                time.sleep(5)
                if attempt == retries - 1:
                    logging.error(f"Skipping {page_url} after {retries} attempts.")
                    return reviews

    return reviews

# Function to process multiple hotels
def extract_top_hotels_reviews(hotel_urls, reviews_per_hotel):
    all_reviews = []
    for idx, hotel_url in enumerate(hotel_urls):
        logging.info(f"Processing hotel {idx+1}/{len(hotel_urls)}: {hotel_url}")
        reviews = extract_hotel_reviews(hotel_url, reviews_per_hotel)
        for review in reviews:
            all_reviews.append({"hotel_url": hotel_url, "review": review})
        logging.info(f"Extracted {len(reviews)} reviews")
        time.sleep(random.uniform(2, 5))
    return all_reviews

# Example hotel URLs (real links needed)
hotel_urls = [
    'https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html',
    'https://www.tripadvisor.com/Hotel_Review-g60898-d86109-Reviews-The_Ritz_Carlton_Atlanta-Atlanta_Georgia.html',
    # Add more URLs as needed
]

# Parameters
reviews_per_hotel = 20  # Adjust as needed
hotel_data = extract_top_hotels_reviews(hotel_urls, reviews_per_hotel)

# Save to Excel
df = pd.DataFrame(hotel_data)
df.to_excel("tripadvisor_hotel_reviews.xlsx", index=False)
logging.info("Saved all reviews to tripadvisor_hotel_reviews.xlsx")


2025-05-13 11:25:23,978 - INFO - Processing hotel 1/2: https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html
2025-05-13 11:25:23,979 - INFO - Fetching: https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html-or0
2025-05-13 11:25:26,214 - WARNING - No reviews found on page https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html-or0
2025-05-13 11:25:26,214 - INFO - Fetching: https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html-or0
2025-05-13 11:25:28,477 - WARNING - No reviews found on page https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_New_York-New_York_City_New_York.html-or0
2025-05-13 11:25:28,478 - INFO - Fetching: https://www.tripadvisor.com/Hotel_Review-g60763-d93540-Reviews-Park_Central_Hotel_N